In [3]:
#what we need: exceptions if player doesn't exist, position, pandas, ohtani?

#below imports everthing we need
import pandas as pd
import numpy as np
import requests
import warnings
import sys
import json


mlb=pd.read_csv('MLB_Walkup.csv')
mlbplayer=input("Enter an active MLB player or type 'quit': ")

data = mlb
df = pd.DataFrame(data)

while True:
    try:
        if mlbplayer == 'quit':
            break        

        row =(df.loc[df['Player'] == mlbplayer])
        row = pd.DataFrame({'Team': row['Team'],'Player': row['Player'], 'Artist': row['Artist'], 'Song': row['Song'], 'Playlist': row['Playlist']})
        row


        def make_clickable(val):
            return '<a target="_blank" href="{}">{}</a>'.format(val, val)

        row.style.format({'Playlist': make_clickable})

        if df['Player'].count == 0:
            print("player doesn't exist")
        else:

            #beginning of mlb api
            url=("http://lookup-service-prod.mlb.com//json/named.search_player_all.bam?sport_code='mlb'&active_sw='Y'&name_part='%s'" % (mlbplayer)) 
            response=requests.get(url)
            response.ok

            #below gets player id for url2 and url3 as well as position to determine whether player is batter or pitcher
            player=response.json()
            playerid=player['search_player_all']['queryResults']['row']['player_id']
            position=player['search_player_all']['queryResults']['row']['position']
            playername=player['search_player_all']['queryResults']['row']['name_display_first_last']

            if position == "P":
                #below is for pitchers
                purl=("http://lookup-service-prod.mlb.com//json/named.sport_pitching_tm.bam?league_list_id='mlb'&game_type='R'&season='2018'&player_id='%s'" % (playerid))
                response3=requests.get(purl)
                response3.ok
                p_stats=response3.json()
                wins=p_stats['sport_pitching_tm']['queryResults']['row']['w']
                innings=p_stats['sport_pitching_tm']['queryResults']['row']['ip']
                strikeouts=p_stats['sport_pitching_tm']['queryResults']['row']['so']
                print("Player", playername, "Position", position, "Wins:", wins, "Innings:", innings, "Strikeouts:", strikeouts)
                break
            
            else:
                
                #below is for batters
                burl=("http://lookup-service-prod.mlb.com//json/named.sport_hitting_tm.bam?league_list_id='mlb'&game_type='R'&season='2018'&player_id='%s'" % (playerid))
                response2=requests.get(burl)
                response2.ok
                b_stats=response2.json()
                average=b_stats['sport_hitting_tm']['queryResults']['row']['avg']
                hr=b_stats['sport_hitting_tm']['queryResults']['row']['hr']
                rbi=b_stats['sport_hitting_tm']['queryResults']['row']['rbi']
                print("Player", playername, "Position", position, "Average:", average, "Home Runs:", hr, "RBI:", rbi)
            

            
            
    except KeyError:
        print("This player does not exist, is spelled wrong, was not active in 2018, or is not in the database of mlb.com/entertainment")
        break


Enter an active MLB player or type 'quit': Sean Doolittle
Player Sean Doolittle Position P Average: .--- Home Runs: 0 RBI: 0
Player Sean Doolittle Position P Wins: 3 Innings: 45.0 Strikeouts: 60
